In [ ]:
import cv2 
import numpy as np 
from matplotlib import pyplot as plt
from pathlib import Path



In [ ]:
jeuDeDonneesLogo = ["Bershka-logo.png","primark-logo.png","zara-logo.png","hm-logo.png","Shein-logo.png"]
jeuDeDonneesImage = ["BERSHKA.jpg","PRIMARK.jpg","ZARA.jpeg","H&M.jpg","SHEIN.jpeg"]

for i in jeuDeDonneesLogo:
    fichier = i
    img = cv2.imread(fichier, cv2.IMREAD_COLOR) 

    cv2.imshow("Titre", img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

for i in jeuDeDonneesImage:
    fichier = i
    img = cv2.imread(fichier, cv2.IMREAD_COLOR) 

    cv2.imshow("Titre", img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
